In [23]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import RFormula
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType

In [2]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [3]:
counters = sqlContext.read.format("org.apache.spark.sql.cassandra").load(table="counters", keyspace="dice")
counters.cache()

DataFrame[id: string, modified: timestamp, contentname: string, crsid: string, description: string, icon: string, stevci_cestaopis: string, stevci_gap: float, stevci_hit: int, stevci_lokacijaopis: string, stevci_pasopis: string, stevci_smeropis: string, stevci_stat: int, stevci_statopis: string, stevci_stev: int, title: string, x: float, x_wgs: float, y: float, y_wgs: float]

In [33]:
hourTokens = udf(lambda time: time.hour, IntegerType())
counters = counters.withColumn('hour', hourTokens(counters.modified))
weekTokens = udf(lambda time: time.weekday(), IntegerType())
counters = counters.withColumn('weekday', weekTokens(counters.modified))

In [60]:
formula = RFormula(formula="stevci_stat ~ id + hour + weekday + hour : weekday + id : hour : weekday")
data = formula.fit(counters).transform(counters)

In [61]:
train, test = data.randomSplit([0.7, 0.3], 1234)

In [62]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
model = nb.fit(train)

In [133]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(maxDepth=20, minInstancesPerNode=4)
model = dt.fit(train)

In [111]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(numTrees=30, minInstancesPerNode=3, maxDepth=20)
model = rf.fit(train)

In [134]:
predictions = model.transform(test)
# predictions.select('stevci_stat', 'label', 'prediction').show()
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.747549019608
